# Introduccion a Web Scraping y APIs publicas con Python

En este taller aprenderemos a obtener datos de internet de forma programatica usando Python.  
Trabajaremos con datos reales del **INE** (Instituto Nacional de Estadistica).

**Contenido:**
1. Como funciona una peticion HTTP (la base de todo)
2. La API JSON del INE: navegar operaciones, tablas y series
3. Descarga directa de CSV (metodo rapido via jaxiT3)
4. Web scraping con BeautifulSoup (para cuando no hay API)
5. Ejemplo completo combinando todo

**Librerias necesarias:**
```
pip install requests beautifulsoup4 pandas lxml
```
Ya Instaladas con requirements.txt

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

---
## Parte 1: Como funciona una peticion HTTP

Cuando escribes una URL en el navegador, este envia una **peticion HTTP** al servidor.  
El servidor responde con un **codigo de estado** y el **contenido** (HTML, CSV, JSON...).

Codigos de estado mas comunes:
- `200` -> OK, todo bien
- `301/302` -> Redireccion (la pagina se ha movido)
- `404` -> No encontrado
- `500` -> Error del servidor

In [ ]:
# Hacemos una peticion a la web del INE
url = 'https://www.ine.es'
respuesta = requests.get(url)

print(f'Codigo de estado: {respuesta.status_code}')
print(f'Tipo de contenido: {respuesta.headers.get("Content-Type", "desconocido")}')
print(f'Tamano de la respuesta: {len(respuesta.text):,} caracteres')

In [ ]:
# El servidor devuelve HTML crudo: el mismo codigo que el navegador renderiza
print(respuesta.text[:500])

Con `requests` podemos hacer lo mismo que hace el navegador, pero de forma programatica.  
La diferencia es que en vez de ver una pagina bonita, recibimos el **texto crudo** (HTML, JSON, CSV...).

---
## Parte 2: La API JSON del INE

El INE ofrece una **API JSON** que permite acceder a todos los datos de INEbase mediante URLs estructuradas.  
Es la forma mas limpia y fiable de obtener datos del INE.

**Estructura de la API:**
```
https://servicios.ine.es/wstempus/js/{idioma}/{funcion}/{input}[?parametros]
```

Donde:
- `{idioma}`: `ES` o `EN`
- `{funcion}`: la accion que queremos (ver tabla abajo)
- `{input}`: el identificador (de operacion, tabla, serie...)

**Jerarquia de la API (de mas general a mas concreto):**
```
Operaciones  ->  Tablas  ->  Series  ->  Datos
  (IPC)        (50902)     (IPC251856)   (valores)
```

**Funciones principales:**

| Funcion | Que devuelve | Ejemplo |
|---|---|---|
| `OPERACIONES_DISPONIBLES` | Todas las operaciones en Tempus3 | Lista de IPC, EPA, etc. |
| `TABLAS_OPERACION/{op}` | Tablas de una operacion | Tablas del IPC |
| `GRUPOS_TABLA/{id}` | Variables/filtros de una tabla | Combos de seleccion |
| `DATOS_TABLA/{id}` | Datos de una tabla | Valores numericos |
| `DATOS_SERIE/{cod}` | Datos de una serie concreta | Serie temporal |

**Documentacion oficial:** https://www.ine.es/dyngs/DAB/index.htm?cid=1099  
**Referencia de funciones:** https://www.ine.es/dyngs/DAB/index.htm?cid=1100  
**Cheatsheet (R, pero mismos conceptos):** https://github.com/es-ine/ineapir

### 2.1 Listar operaciones disponibles

El punto de entrada es `OPERACIONES_DISPONIBLES`: devuelve todas las operaciones estadisticas del INE.

In [ ]:
# Base URL de la API
BASE = 'https://servicios.ine.es/wstempus/js/ES'

# Listar todas las operaciones disponibles
r = requests.get(f'{BASE}/OPERACIONES_DISPONIBLES')
operaciones = r.json()

len(operaciones)

In [ ]:
print(f'Total operaciones disponibles: {len(operaciones)}')
print(f'\nPrimeras 5:')
for op in operaciones[105:]:
    print(f"  Id={op['Id']:>2}  Codigo={op['Codigo']:<10}  {op['Nombre']}")

In [ ]:
# Convertimos a DataFrame para buscar mas comodamente
df_ops = pd.DataFrame(operaciones)
df_ops.head(10)

In [ ]:
df_ops.convert_dtypes().dtypes

In [ ]:
# Buscar operaciones por nombre (ejemplo: buscar todo lo que contenga 'precio')
#
# str.contains() busca un texto dentro de cada celda de la columna.
# Parametros:
#   case=False -> no distinguir mayusculas/minusculas ('recio' encuentra 'Precio', 'PRECIO'...)
#   case=True  -> busqueda estricta (el defecto)
#   na=False   -> si una celda es NaN (vacia), tratarla como False (no coincide)
#                 sin esto, las celdas NaN darian NaN en vez de True/False y causarian error

mascara = df_ops['Nombre'].str.contains('recio', case=False, na=False)
df_ops.loc[mascara, ['Id', 'Codigo', 'Nombre']]

### 2.2 Listar tablas de una operacion

Una vez identificada la operacion, listamos sus tablas con `TABLAS_OPERACION`.  
Cada tabla tiene un **Id** que necesitaremos para descargar los datos.

La operacion se puede identificar de tres formas equivalentes:
- Por Id numerico: `25`
- Por codigo IOE: `IOE30138`  
- Por codigo corto: `IPC`

In [ ]:
# Listar tablas del IPC (Indice de Precios de Consumo)
r = requests.get(f'{BASE}/TABLAS_OPERACION/IPC')
tablas_ipc = r.json()

print(f'El IPC tiene {len(tablas_ipc)} tablas')
print()
for t in tablas_ipc[:10]:
    print(f"  Id={t['Id']:<8} {t['Nombre']}")

In [ ]:
# Tambien podemos convertir a DataFrame para explorar
df_tablas = pd.DataFrame(tablas_ipc)
df_tablas[['Id', 'Nombre', 'Anyo_Periodo_ini']].head(15)

### 2.3 Obtener datos de una tabla

Con el Id de la tabla, usamos `DATOS_TABLA` para obtener los datos.  

**Pero antes:** ¿como sabemos que parametros acepta? La documentacion de la API esta aqui:  
https://www.ine.es/dyngs/DAB/index.htm?cid=1100
https://www.ine.es/OpenAPI/includes/files/en/wstempus.yaml

**Parametros principales** (todos opcionales, se anaden a la URL con `?param=valor`):
- `nult=N` -> solo los ultimos N periodos. **Muy recomendable** para explorar, porque sin el devuelve TODO el historico y la respuesta puede ser enorme
- `tip=A` -> formato amigable (fechas legibles, nombres en vez de codigos)
- `tip=M` -> incluir metadatos (unidades, etc.)
- `tip=AM` -> ambos: amigable + metadatos
- `det=0|1|2` -> nivel de detalle (0=minimo, 2=maximo)
- `tv=variable:valor` -> filtrar (lo veremos en la seccion 2.5)

Empecemos explorando la estructura del JSON que devuelve la API:

In [ ]:
# Descargar datos de la tabla 50902 (Indices nacionales: general y de grupos ECOICOP)
# Pedimos solo los ultimos 2 periodos para no saturar
r = requests.get(f'{BASE}/DATOS_TABLA/76116', params={'nult': 4})
datos_json = r.json()

# ¿Que es datos_json? Una LISTA de series. Cada serie es un diccionario.
print(f'Tipo: {type(datos_json)}  ->  lista de {len(datos_json)} series')
print(f'\n--- Primera serie (cruda) ---')
print(f'Claves: {list(datos_json[0].keys())}')
print()

serie_0 = datos_json[0]
print(f"  COD:        {serie_0['COD']}")
print(f"  Nombre:     {serie_0['Nombre']}")
print(f"  FK_Unidad:  {serie_0['FK_Unidad']}   <- codigo de la unidad (133=indice, 135=porcentaje...)")
print(f"  FK_Escala:  {serie_0['FK_Escala']}")
print(f"  Data:       lista de {len(serie_0['Data'])} periodos")
print(f'\n--- Un dato individual ---')
print(serie_0['Data'][3])

In [ ]:
# Estructura de un dato individual
# La Fecha viene en milisegundos desde epoch (Unix timestamp * 1000)
dato = serie_0['Data'][0]
print(f'Dato crudo: {dato}')
print(f'\nFecha epoch (ms): {dato["Fecha"]}')
print(f'Fecha legible:    {datetime.fromtimestamp(dato["Fecha"]/1000).strftime("%Y-%m-%d")}')
print(f'Periodo:          {dato["FK_Periodo"]} (mes)')
print(f'Ano:              {dato["Anyo"]}')
print(f'Valor:            {dato["Valor"]}')

**Resumen de la estructura JSON:**
```
datos_json = [           <- lista de series
  {
    'COD': 'IPC251852',  <- codigo unico de la serie
    'Nombre': '...',     <- descripcion
    'FK_Unidad': 133,    <- unidad (133=indice, 135=porcentaje, etc.)
    'FK_Escala': 1,      <- escala
    'Data': [            <- lista de observaciones
      {
        'Fecha': 1764...,    <- epoch en milisegundos
        'FK_Periodo': 12,    <- mes
        'Anyo': 2025,        <- ano
        'Valor': 119.942,    <- el dato
        'FK_TipoDato': 1,    <- tipo (1=definitivo, 2=provisional...)
        'Secreto': false     <- si el dato es confidencial
      }, ...
    ]
  }, ...
]
```

Los codigos como `FK_Unidad=133` no son muy legibles. Con `tip=AM` la API incluye nombres en vez de codigos:

In [ ]:
# Con tip=AM obtenemos nombres legibles en vez de codigos numericos
r = requests.get(f'{BASE}/DATOS_TABLA/50902', params={'nult': 1, 'tip': 'AM'})
datos_am = r.json()

serie = datos_am[0]
print(f"Nombre:  {serie['Nombre']}")
#print(f"Unidad:  {serie['Unidad']}")
print(f"\nDato:")
d = serie['Data'][0]
for clave, valor in d.items():
    print(f"  {clave}: {valor}")

In [ ]:
# Convertir a DataFrame: aplanar la estructura JSON anidada
# (volvemos a pedir con nult=5 para tener mas datos)
r = requests.get(f'{BASE}/DATOS_TABLA/50902', params={'nult': 5})
datos_json = r.json()

filas = []
for serie in datos_json:
    for d in serie['Data']:
        filas.append({
            'serie': serie['Nombre'].strip(),
            'codigo': serie['COD'],
            'periodo': d['FK_Periodo'],
            'anyo': d['Anyo'],
            'valor': d['Valor']
        })

df = pd.DataFrame(filas)
print(f'DataFrame: {df.shape[0]} filas x {df.shape[1]} columnas')
df.head(10)

In [ ]:
# Filtrar solo la variacion anual del indice general
mascara = df['serie'].str.contains('general.*anual', case=False)
df_anual = df[mascara].copy()
df_anual

### 2.4 Obtener datos de una serie concreta

Si ya conoces el codigo de la serie (ej: `IPC251856` = variacion anual del indice general),  
puedes pedir directamente sus datos con `DATOS_SERIE`.

In [ ]:
# Descargar datos de una serie concreta
r = requests.get(f'{BASE}/DATOS_SERIE/IPC251856', params={'nult': 12})
serie_data = r.json()

# Convertir a DataFrame
df_serie = pd.DataFrame(serie_data['Data'])
df_serie['fecha'] = pd.to_datetime(df_serie['Fecha'], unit='ms')
df_serie = df_serie[['fecha', 'Anyo', 'FK_Periodo', 'Valor']]
df_serie.columns = ['fecha', 'anyo', 'mes', 'valor']

print(f"Serie: {serie_data['Nombre']}")
df_serie

### 2.5 Filtrar datos con el parametro `tv`

Las tablas grandes se pueden filtrar por variables y valores usando el parametro `tv`.  
El formato es: `tv=id_variable:id_valor`

Para conocer las variables disponibles de una tabla, usamos `GRUPOS_TABLA`.

In [ ]:
# Ver los grupos (variables de filtro) de la tabla 50913
# (Indices por comunidades autonomas: general y grupos ECOICOP)
r = requests.get(f'{BASE}/GRUPOS_TABLA/50913')
grupos = r.json()

for g in grupos:
    print(f"Grupo Id={g['Id']}  -> {g['Nombre']}")

In [ ]:
# Ver los valores posibles de un grupo
# Ejemplo: ver las comunidades autonomas disponibles
grupo_id = grupos[0]['Id']  # primer grupo (comunidades autonomas)
r = requests.get(f'{BASE}/VALORES_GRUPOSTABLA/50913/{grupo_id}')
valores = r.json()

print(f'Valores del grupo "{grupos[0]["Nombre"]}":')
for v in valores[:10]:
    print(f"  Id={v['Id']:<8} {v['Nombre']}")

### 2.6 Como obtener los IDs

El mayor reto de la API es conocer los IDs de tablas. Se puede hacer a través de:

**1. Desde la URL de INEbase**:  
En la web del INE, cada tabla tiene una URL tipo:  
- `ine.es/jaxiT3/Tabla.htm?t=50902` -> el Id de tabla es **50902** (tabla Tempus3)  
- `ine.es/jaxi/Tabla.htm?path=/t20/e245/p08/l0/&file=01001.px` -> Id: **t20/e245/p08/l0/01001.px** (PC-Axis)
- `ine.es/jaxi/Tabla.htm?tpx=33387` -> Id: **33387** (tabla tpx)

**2. Via API** (lo que hemos hecho):  
`OPERACIONES_DISPONIBLES` -> `TABLAS_OPERACION/{op}` -> `GRUPOS_TABLA/{id}` -> `VALORES_GRUPOSTABLA/{id}/{grupo}`

### 2.7 Funcion generica para descargar datos del INE

Juntamos todo en una funcion reutilizable:

In [ ]:
import requests
import pandas as pd


def descargar_tabla_ine_json(id_tabla, nult=None, tip=None, filtros=None):
    base = 'https://servicios.ine.es/wstempus/js/ES'
    params = {}
    if nult:
        params['nult'] = nult
    if tip:
        params['tip'] = tip
    if filtros:
        params['tv'] = filtros

    r = requests.get(f'{base}/DATOS_TABLA/{id_tabla}', params=params)
    r.raise_for_status()
    datos = r.json()

    filas = []
    for serie in datos:
        for d in serie.get('Data', []):
            filas.append({
                'serie': serie['Nombre'].strip(),
                'codigo': serie['COD'],
                'periodo': d['FK_Periodo'],
                'anyo': d['Anyo'],
                'valor': d['Valor']
            })

    return pd.DataFrame(filas)


if __name__ == '__main__':
    df = descargar_tabla_ine_json(50902, nult=3)
    print(f'{len(df)} filas descargadas')
    df.to_csv('ipc_datos.csv', index=False, sep=';')
    print(f'Guardado ipc_datos.csv con {len(df)} filas')

---
## Parte 3: Descarga directa de CSV (el atajo via jaxiT3)

Para tablas Tempus3, el INE tambien ofrece descarga directa en CSV:  
```
https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/{id_tabla}.csv?nocab=1
```

Esto es mas simple que la API JSON, pero menos flexible (no permite filtrar).  
Es ideal para descargas rapidas cuando quieres toda la tabla.

In [ ]:
# Descargar tabla 2879 (poblacion por municipios, sexo y edad)
url_csv = 'https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/2879.csv?nocab=1'
df_csv = pd.read_csv(url_csv, sep=';', encoding='utf-8')

print(f'Tabla descargada: {df_csv.shape[0]} filas x {df_csv.shape[1]} columnas')
print(f'Columnas: {list(df_csv.columns)}')
df_csv.head()

In [ ]:
# Funcion para descarga rapida via CSV
def descargar_tabla_ine_csv(id_tabla, idioma='es'):
    """
    Descarga una tabla del INE como CSV y devuelve un DataFrame.
    Solo funciona con tablas Tempus3 (ids numericos).
    """
    url = f'https://www.ine.es/jaxiT3/files/t/{idioma}/csv_bdsc/{id_tabla}.csv?nocab=1'
    r = requests.get(url)
    r.raise_for_status()
    
    from io import StringIO
    return pd.read_csv(StringIO(r.text), sep=';', encoding='utf-8')


# Ejemplo
df = descargar_tabla_ine_csv(30664)
print(f'Descargada: {df.shape}')
df.head()

### Cuando usar cada metodo

| Metodo | Ventajas | Inconvenientes |
|---|---|---|
| **API JSON** (`DATOS_TABLA`) | Filtros, paginacion, solo ultimos N periodos | Requiere aplanar JSON |
| **CSV** (`jaxiT3`) | Directo a pandas, mas simple | Sin filtros, descarga tabla completa |
| **Web scraping** (HTML) | Para datos sin API | Fragil, depende de la estructura HTML |

---
## Parte 4: Web Scraping con BeautifulSoup

No siempre los datos estan disponibles via API o CSV descargable.  
A veces necesitamos **extraer datos directamente del HTML** de una pagina web.

**El proceso es siempre:**
1. Descargar el HTML con `requests.get()`
2. Parsear el HTML con `BeautifulSoup`
3. Buscar los elementos que contienen los datos
4. Extraer el texto o atributos

In [ ]:
# Ejemplo: obtener la lista de operaciones estadisticas del INE
# (scraping del catalogo HTML, no de la API)

url_operaciones = 'https://www.ine.es/dyngs/INEbase/listaoperaciones.htm'
respuesta = requests.get(url_operaciones)

print(f'Status: {respuesta.status_code}')
print(f'Tamano: {len(respuesta.text):,} caracteres')

In [ ]:
# Parsear el HTML con BeautifulSoup
# - Primer argumento: el texto HTML
# - Segundo argumento: el parser a usar ('html.parser' viene con Python, 'lxml' es mas rapido)
soup = BeautifulSoup(respuesta.text, 'html.parser')

# Encontrar todos los enlaces <a>
todos_los_enlaces = soup.find_all('a')
print(f'Total enlaces en la pagina: {len(todos_los_enlaces)}')

In [ ]:
# Metodos principales de busqueda de BeautifulSoup:
#
#   soup.find('etiqueta')              -> primer elemento con esa etiqueta
#   soup.find_all('etiqueta')          -> TODOS los elementos (lista)
#   soup.find('a', class_='mi-clase')  -> filtrar por atributo CSS
#   soup.find('a', href=True)          -> que tenga el atributo href
#   soup.select('div.clase > a')       -> selectores CSS (como en el navegador)

# Filtrar solo los enlaces que apuntan a operaciones estadisticas
operaciones_html = [
    {'nombre': a.text.strip(), 'url': a['href']}
    for a in todos_los_enlaces
    if a.get('href', '').startswith('/dyngs/INEbase/')
    and a.text.strip()
]

print(f'Operaciones encontradas: {len(operaciones_html)}')
for op in operaciones_html[:10]:
    print(f"  {op['nombre'][:60]:<20} -> {op['url'][:50]}")

---
## Parte 5: Ejemplo completo — Indicadores demograficos por municipio

Combinamos todo lo aprendido:
1. API JSON para descubrir las tablas disponibles
2. Descarga de datos (JSON o CSV)
3. Limpieza y transformacion con pandas
4. Guardado del resultado

In [ ]:
# Paso 1: buscar tablas de la operacion 'Indicadores Demograficos Basicos' (IDB)
r = requests.get(f'{BASE}/TABLAS_OPERACION/IDB')
tablas_idb = r.json()

print(f'Tablas de Indicadores Demograficos Basicos ({len(tablas_idb)}):')
for t in tablas_idb[:15]:
    print(f"  Id={t['Id']:<8} {t['Nombre']}")

In [ ]:
# Paso 2: descargar varias tablas de indicadores por municipio
# Usamos el metodo CSV (mas simple para tablas completas)
tablas_interes = {
    '30664': 'Tasa bruta de natalidad',
    '30665': 'Tasa bruta de mortalidad',
    '30681': 'Edad media de la poblacion',
}

dataframes = {}
for id_tabla, nombre in tablas_interes.items():
    print(f'Descargando: {nombre} (tabla {id_tabla})...')
    url = f'https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/{id_tabla}.csv?nocab=1'
    df = pd.read_csv(url, sep=';', encoding='utf-8')
    dataframes[nombre] = df
    print(f'  -> {df.shape[0]} filas, columnas: {list(df.columns)}')

print('\nDescarga completada!')

In [ ]:
# Paso 3: explorar una tabla para entender la estructura
df_ejemplo = dataframes['Tasa bruta de natalidad']
print('Columnas:', list(df_ejemplo.columns))
print()
for col in df_ejemplo.columns:
    n_unicos = df_ejemplo[col].nunique()
    ejemplo = df_ejemplo[col].iloc[0]
    print(f'  {col:<30} {n_unicos:>6} valores unicos  (ej: {ejemplo})')

In [ ]:
# Paso 4: limpiar y combinar las tablas

def limpiar_tabla_ine(df, nombre_indicador):
    """Limpia un CSV del INE: ultimo periodo, separa codigo de municipio."""
    # Quedarnos con el periodo mas reciente
    col_periodo = [c for c in df.columns if 'periodo' in c.lower() or 'Periodo' in c][0]
    ultimo = df.groupby(col_periodo).size().index[-1]
    df_ult = df[df[col_periodo] == ultimo].copy()
    
    # Buscar columna de municipio
    col_muni = [c for c in df_ult.columns if 'unici' in c.lower()][0]
    
    # Separar codigo y nombre del municipio
    df_ult['cod_municipio'] = df_ult[col_muni].str.extract(r'^(\d+)')
    df_ult['municipio'] = df_ult[col_muni].str.replace(r'^\d+\s*', '', regex=True).str.strip()
    
    # Buscar columna de valor (Total)
    col_valor = [c for c in df_ult.columns if 'total' in c.lower() or 'Total' in c]
    if col_valor:
        df_ult['valor'] = pd.to_numeric(
            df_ult[col_valor[0]].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False),
            errors='coerce'
        )
    
    df_ult['indicador'] = nombre_indicador
    return df_ult[['cod_municipio', 'municipio', 'indicador', 'valor']].dropna()


# Aplicar a todas las tablas
resultados = []
for nombre, df in dataframes.items():
    try:
        df_limpio = limpiar_tabla_ine(df, nombre)
        resultados.append(df_limpio)
        print(f'{nombre}: {len(df_limpio)} municipios')
    except Exception as e:
        print(f'{nombre}: ERROR - {e}')

resultado_final = pd.concat(resultados, ignore_index=True)
print(f'\nTotal: {len(resultado_final)} filas')
resultado_final.head(10)

In [ ]:
# Paso 5: pivotar para tener un indicador por columna
tabla_pivot = resultado_final.pivot_table(
    index=['cod_municipio', 'municipio'],
    columns='indicador',
    values='valor'
).reset_index()

tabla_pivot.head(10)

In [ ]:
# Paso 6: guardar en disco
tabla_pivot.to_csv('indicadores_municipales.csv', index=False, sep=';')
print('Guardado en indicadores_municipales.csv')

---
## Resumen

| Que quiero hacer | Herramienta | Ejemplo |
|---|---|---|
| Descargar datos del INE (con filtros) | `requests` + API JSON | `DATOS_TABLA/50902?nult=5` |
| Descargar tabla completa del INE | `requests` + CSV jaxiT3 | `jaxiT3/files/t/es/csv_bdsc/{id}.csv` |
| Extraer datos de HTML | `BeautifulSoup` | `soup.find_all('a')` |
| Leer tablas HTML | `pd.read_html()` | `pd.read_html(url)` |
| Limpiar y transformar datos | `pandas` | `df.groupby()`, `df.pivot_table()` |

### API JSON del INE — referencia rapida

| Funcion | URL | Descripcion |
|---|---|---|
| Operaciones | `.../OPERACIONES_DISPONIBLES` | Lista todas las operaciones estadisticas |
| Tablas | `.../TABLAS_OPERACION/{op}` | Tablas de una operacion |
| Grupos | `.../GRUPOS_TABLA/{id}` | Variables de filtro de una tabla |
| Valores | `.../VALORES_GRUPOSTABLA/{id}/{grupo}` | Valores posibles de un grupo |
| Datos tabla | `.../DATOS_TABLA/{id}[?nult=N&tip=A]` | Datos de una tabla |
| Datos serie | `.../DATOS_SERIE/{cod}[?nult=N]` | Datos de una serie concreta |
| Variables op. | `.../VARIABLES_OPERACION/{op}` | Variables de una operacion |
| Valores var. | `.../VALORES_VARIABLEOPERACION/{var}/{op}` | Valores de variable en operacion |

Base: `https://servicios.ine.es/wstempus/js/ES`

### Recursos

- Documentacion API JSON del INE: https://www.ine.es/dyngs/DAB/index.htm?cid=1099
- Referencia de funciones API: https://www.ine.es/dyngs/DAB/index.htm?cid=1100
- Obtener datos de una tabla: https://www.ine.es/dyngs/DAB/index.htm?cid=1102
- Codigos identificadores de tablas: https://www.ine.es/dyngs/DAB/index.htm?cid=1104
- Paquete ineapir (R): https://github.com/es-ine/ineapir
- Catalogo de operaciones INEbase: https://www.ine.es/dyngs/INEbase/listaoperaciones.htm
- Calendario de publicaciones: https://www.ine.es/daco/daco41/calen.htm
- Documentacion de requests: https://docs.python-requests.org
- Documentacion de BeautifulSoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
- Documentacion de pandas: https://pandas.pydata.org/docs/